In [1]:
from ultralytics import YOLO
import cv2

import util
from sort.sort import *
from util import get_car, read_license_plate, write_csv

Using CPU. Note: This module is much faster with a GPU.


In [2]:
!git clone https://github.com/abewley/sort.git

fatal: destination path 'sort' already exists and is not an empty directory.


In [2]:
results = {}

mot_tracker = Sort()

In [3]:
coco_model = YOLO('yolov8n.pt')
license_plate_detector = YOLO('license_plate_detector.pt')

In [13]:
!git clone https://github.com/computervisioneng/train-yolov8-custom-dataset-step-by-step-guide.git

Cloning into 'train-yolov8-custom-dataset-step-by-step-guide'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (34/34), done.
remote: Total 48 (delta 13), reused 39 (delta 11), pack-reused 2 (from 1)
Receiving objects: 100% (48/48), 23.85 KiB | 139.00 KiB/s, done.
Resolving deltas: 100% (13/13), done.


In [5]:
# load video
cap = cv2.VideoCapture('./sample.mp4')

vehicles = [2, 3, 5, 7]

In [6]:
# read frames
frame_nmr = -1
ret = True
while ret:
    frame_nmr += 1
    ret, frame = cap.read()
    if ret:
        results[frame_nmr] = {}
        # detect vehicles
        detections = coco_model(frame)[0]
        detections_ = []
        for detection in detections.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = detection
            if int(class_id) in vehicles:
                detections_.append([x1, y1, x2, y2, score])

        # track vehicles
        track_ids = mot_tracker.update(np.asarray(detections_))

        # detect license plates
        license_plates = license_plate_detector(frame)[0]
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            # assign license plate to car
            xcar1, ycar1, xcar2, ycar2, car_id = get_car(license_plate, track_ids)

            if car_id != -1:

                # crop license plate
                license_plate_crop = frame[int(y1):int(y2), int(x1): int(x2), :]

                # process license plate
                license_plate_crop_gray = cv2.cvtColor(license_plate_crop, cv2.COLOR_BGR2GRAY)
                _, license_plate_crop_thresh = cv2.threshold(license_plate_crop_gray, 64, 255, cv2.THRESH_BINARY_INV)

                # read license plate number
                license_plate_text, license_plate_text_score = read_license_plate(license_plate_crop_thresh)

                if license_plate_text is not None:
                    results[frame_nmr][car_id] = {'car': {'bbox': [xcar1, ycar1, xcar2, ycar2]},
                                                  'license_plate': {'bbox': [x1, y1, x2, y2],
                                                                    'text': license_plate_text,
                                                                    'bbox_score': score,
                                                                    'text_score': license_plate_text_score}}

# write results
write_csv(results, './test.csv')


0: 384x640 21 cars, 1 bus, 2 trucks, 49.8ms
Speed: 2.0ms preprocess, 49.8ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 43.4ms
Speed: 1.3ms preprocess, 43.4ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 23 cars, 1 bus, 2 trucks, 40.5ms
Speed: 3.9ms preprocess, 40.5ms inference, 0.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 34.7ms
Speed: 1.5ms preprocess, 34.7ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 40.7ms
Speed: 1.5ms preprocess, 40.7ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 42.5ms
Speed: 1.0ms preprocess, 42.5ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 1 truck, 34.9ms
Speed: 1.3ms preprocess, 34.9ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 license_plates, 3

In [7]:
%run add_missing_data.py

['104', '110', '115', '116', '120', '121', '122', '124', '126', '127', '132', '133', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157', '158', '159', '160', '161', '162', '163', '164', '165', '166', '167', '170', '172', '174', '175', '176', '177', '178', '179', '180', '181', '182', '183', '184', '185', '186', '189', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', '200', '201', '202', '203', '206', '207', '208', '209', '210', '211', '212', '213', '215', '216', '217', '218'] 1
['2', '8', '9', '10', '12', '13', '14', '15', '16', '18', '19', '21', '22', '23', '25', '26', '28', '30'] 3
['2', '9', '12', '15', '20', '25', '31', '40', '41', '50', '55', '58', '59', '62'] 5
['43', '56', '57', '58', '64', '80', '86', '87', '90', '92', '93', '94', '96', '99', '100', '102', '103', '120', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '136'] 6
['118', '1

In [10]:
%run visualize.py